<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/ThaiNovel_OCR_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
================================================================================
                OCR PROCESSING WITH API v3.0 - FIXED VERSION
                       Enhanced Thai Novel OCR Processor
================================================================================
"""

# ============================================
# 📌 Block 1: Setup & Import
# ============================================
import os
import re
import json
import time
import random
import unicodedata
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple, Optional
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Mount Drive (สำหรับ Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

print("✅ Libraries loaded")

Mounted at /content/drive
✅ Libraries loaded


In [2]:
# ============================================
# 📌 Block 2: API Key Loading & Configuration
# ============================================

def load_api_key():
    """โหลด API key จากไฟล์ (รองรับทั้ง .env และ .env.txt)"""
    base_path = "/content/drive/MyDrive/OCR" if IN_COLAB else "./OCR"

    # ลองหาไฟล์ทั้งสองแบบ
    possible_files = [
        f"{base_path}/openai.env",
        f"{base_path}/openai.env.txt"
    ]

    for env_path in possible_files:
        try:
            # อ่านค่า key
            with open(env_path, "r") as f:
                key = f.read().strip()

            if key and key.startswith('sk-'):
                # ตั้ง environment variable
                os.environ["OPENAI_API_KEY"] = key
                print(f"✅ OpenAI API key loaded from: {Path(env_path).name}")
                return key
            else:
                print(f"⚠️ Invalid API key format in {Path(env_path).name}")
                continue

        except FileNotFoundError:
            continue
        except Exception as e:
            print(f"❌ Error reading {env_path}: {e}")
            continue

    # ไม่พบไฟล์ใด ๆ
    print("❌ ไม่พบไฟล์ API key หรือรูปแบบไม่ถูกต้อง")
    print("   กรุณาสร้างไฟล์ใดไฟล์หนึ่ง:")
    for path in possible_files:
        print(f"   - {path}")
    print("   เนื้อหา: sk-xxxxxxxxxxxxxxxxxxxxxxxx")
    return None

class Config:
    """Configuration สำหรับ API Processing"""

    # โหลด API key จากไฟล์
    OPENAI_API_KEY = load_api_key()
    ANTHROPIC_API_KEY = ""  # ใส่ Anthropic API key ถ้าใช้ Claude

    # เลือก Model
    MODEL = "gpt-4o-mini"  # ถูกสุด แนะนำ!
    # MODEL = "gpt-3.5-turbo"
    # MODEL = "claude-3-haiku"

    # Paths (Google Drive)
    BASE = '/content/drive/MyDrive/OCR' if IN_COLAB else './OCR'

    RAW_OCR_DIR = f'{BASE}/raw_ocr'
    CLEANED_DIR = f'{BASE}/cleaned'
    CORPUS_DIR = f'{BASE}/final_corpus'
    TRAINING_PAIRS_DIR = f'{BASE}/training_pairs'
    LOGS_DIR = f'{BASE}/logs'

    # Processing settings
    MAX_PAGES_PER_BATCH = 8  # ลดลงเล็กน้อยเพื่อความเสถียร
    MAX_RETRIES = 3
    TEMPERATURE = 0.05  # ลดลงให้ consistent มากขึ้น
    MAX_TOKENS = 8000
    CONTEXT_OVERLAP = 100  # ตัวอักษรที่ overlap ระหว่าง chunk

    # Cost tracking (แม่นยำขึ้น - แยก input/output)
    PRICE_PER_1K_TOKENS = {
        'gpt-4o-mini': {'input': 0.00015, 'output': 0.0006},  # $0.15/$0.60 per 1M
        'gpt-3.5-turbo': {'input': 0.0005, 'output': 0.0015},
        'claude-3-haiku': {'input': 0.00025, 'output': 0.00125}
    }

# สร้าง folders
for folder in [Config.RAW_OCR_DIR, Config.CLEANED_DIR, Config.CORPUS_DIR,
               Config.TRAINING_PAIRS_DIR, Config.LOGS_DIR]:
    Path(folder).mkdir(parents=True, exist_ok=True)

print("✅ Config loaded")

✅ OpenAI API key loaded from: openai.env
✅ Config loaded


In [3]:
# ============================================
# 📌 Block 3: Usage Logger
# ============================================
class UsageLogger:
    """บันทึกการใช้งาน API เป็น CSV พร้อม detailed tracking"""

    def __init__(self):
        self.log_file = Path(Config.LOGS_DIR) / "usage.csv"
        self.filename_map_file = Path(Config.LOGS_DIR) / "filename_mapping.json"
        self._init_csv()

    def _init_csv(self):
        """สร้างไฟล์ CSV ถ้ายังไม่มี"""
        if not self.log_file.exists():
            columns = [
                'timestamp', 'original_filename', 'clean_filename', 'pages_count',
                'model', 'input_tokens', 'output_tokens', 'total_tokens',
                'cost_usd', 'cost_thb', 'processing_time_sec', 'retry_count',
                'validation_status'
            ]
            df = pd.DataFrame(columns=columns)
            df.to_csv(self.log_file, index=False, encoding='utf-8')

    def log_usage(self, original_filename: str, clean_filename: str, pages_count: int,
                  model: str, input_tokens: int, output_tokens: int, cost_usd: float,
                  processing_time: float = 0, retry_count: int = 0,
                  validation_status: str = 'PASS'):
        """บันทึกการใช้งาน"""

        new_row = {
            'timestamp': datetime.now().isoformat(),
            'original_filename': original_filename,
            'clean_filename': clean_filename,
            'pages_count': pages_count,
            'model': model,
            'input_tokens': input_tokens,
            'output_tokens': output_tokens,
            'total_tokens': input_tokens + output_tokens,
            'cost_usd': cost_usd,
            'cost_thb': cost_usd * 35,  # ประมาณ
            'processing_time_sec': processing_time,
            'retry_count': retry_count,
            'validation_status': validation_status
        }

        # เพิ่มลง CSV
        df = pd.DataFrame([new_row])
        df.to_csv(self.log_file, mode='a', header=False, index=False, encoding='utf-8')

        # บันทึก filename mapping
        self._save_filename_mapping(original_filename, clean_filename)

    def _save_filename_mapping(self, original: str, cleaned: str):
        """บันทึก mapping ระหว่างชื่อไฟล์เดิมและใหม่"""
        mapping = {}
        if self.filename_map_file.exists():
            try:
                with open(self.filename_map_file, 'r', encoding='utf-8') as f:
                    mapping = json.load(f)
            except:
                pass

        mapping[cleaned] = {
            'original': original,
            'timestamp': datetime.now().isoformat()
        }

        with open(self.filename_map_file, 'w', encoding='utf-8') as f:
            json.dump(mapping, f, ensure_ascii=False, indent=2)

    def get_summary(self) -> Dict:
        """สรุปการใช้งานทั้งหมด"""
        try:
            df = pd.read_csv(self.log_file, encoding='utf-8')

            if df.empty:
                return {'total_files': 0, 'total_cost_usd': 0, 'total_tokens': 0}

            return {
                'total_files': len(df),
                'total_pages': df['pages_count'].sum(),
                'total_tokens': df['total_tokens'].sum(),
                'input_tokens': df['input_tokens'].sum(),
                'output_tokens': df['output_tokens'].sum(),
                'total_cost_usd': df['cost_usd'].sum(),
                'total_cost_thb': df['cost_thb'].sum(),
                'avg_cost_per_page': df['cost_usd'].sum() / df['pages_count'].sum() if df['pages_count'].sum() > 0 else 0,
                'avg_processing_time': df['processing_time_sec'].mean(),
                'most_used_model': df['model'].mode()[0] if len(df) > 0 else 'N/A',
                'validation_stats': df['validation_status'].value_counts().to_dict() if 'validation_status' in df.columns else {}
            }
        except Exception as e:
            print(f"Warning: Error reading usage log: {e}")
            return {'total_files': 0, 'total_cost_usd': 0, 'total_tokens': 0}

print("✅ Enhanced Usage Logger ready")

✅ Enhanced Usage Logger ready


In [4]:
# ============================================
# 📌 Block 4: Thai Text Utilities
# ============================================
class ThaiTextUtils:
    """Utilities สำหรับข้อความภาษาไทย"""

    @staticmethod
    def normalize_unicode(text: str) -> str:
        """Normalize Unicode characters"""
        # กำจัด zero-width characters
        text = re.sub(r'[\u200b\ufeff\u00a0]', '', text)
        # Normalize Unicode form
        text = unicodedata.normalize('NFC', text)
        # จัดระเบียบวรรณยุกต์ไทย (ถ้าจำเป็น)
        text = re.sub(r'([ก-ฮ])([ิีึืุู])([่้๊๋])', r'\1\3\2', text)
        return text

    @staticmethod
    def count_thai_quotes(text: str) -> Dict[str, int]:
        """นับจำนวน 'คู่' ของเครื่องหมายคำพูด (pairs)"""
        eng_double = text.count('"') // 2
        eng_single = text.count("'") // 2
        thai_double = (text.count('\u201c') + text.count('\u201d')) // 2  # “ ”
        thai_single = (text.count('\u2018') + text.count('\u2019')) // 2  # ‘ ’
        return {
            'english_double': eng_double,
            'english_single': eng_single,
            'thai_double': thai_double,
            'thai_single': thai_single,
        }

    @staticmethod
    def fix_yamok_spacing(text: str) -> str:
        """แก้ปัญหาการใช้ ๆ (ไม้ยมก)"""
        text = re.sub(r'\s+ๆ\s+', 'ๆ ', text)
        text = re.sub(r'\s+ๆ(?=\s|$)', 'ๆ', text)
        return text

    @staticmethod
    def detect_word_breaks(text: str) -> List[str]:
        """ตรวจหาคำที่แตกหัก (เช่น 'มา กำลัง')"""
        broken_patterns = [
            r'([ก-ฮ])\s+([ก-ฮิีึืุูํ์่้๊๋])',      # พยัญชนะ + เว้นวรรค + สระ/วรรณยุกต์
            r'([ก-ฮิีึืุู])\s+([ก-ฮ]{1}(?![ก-ฮ]))', # คำสั้น ๆ ที่แยก
        ]
        issues = []
        for pattern in broken_patterns:
            matches = re.findall(pattern, text)
            issues.extend([f"{m[0]} {m[1]}" for m in matches])
        return list(set(issues))

print("✅ Thai Text Utilities ready")

✅ Thai Text Utilities ready


In [5]:
# ============================================
# 📌 Block 5: Multi-page Parser (XML Enhanced)
# ============================================
class MultiPageParser:
    """แยกไฟล์หลายหน้าพร้อม XML processing"""

    @staticmethod
    def parse_multipage_file(content: str) -> List[Dict]:
        """แยกไฟล์ที่มีหลายหน้าออกเป็นรายการ"""
        pages = []
        page_sections = re.split(r'--- PAGE:\s*(\d+)\s*---', content)
        if len(page_sections) > 1:
            for i in range(1, len(page_sections), 2):
                if i + 1 < len(page_sections):
                    page_num = int(page_sections[i])
                    page_content = page_sections[i + 1]
                    raw_text = ""
                    cleaned_text = ""
                    raw_match = re.search(
                        r'--- RAW ---(.*?)(?=--- CLEANED ---|--- PAGE:|$)',
                        page_content, re.DOTALL
                    )
                    if raw_match:
                        raw_text = raw_match.group(1)  # keep as-is
                    else:
                        raw_text = page_content
                    cleaned_match = re.search(
                        r'--- CLEANED ---(.*?)(?=--- PAGE:|$)',
                        page_content, re.DOTALL
                    )
                    if cleaned_match:
                        cleaned_text = cleaned_match.group(1).strip()
                        cleaned_text = ThaiTextUtils.normalize_unicode(cleaned_text)
                    pages.append({
                        'page_num': page_num,
                        'raw_text': raw_text,
                        'cleaned_text': cleaned_text
                    })
        return pages

    @staticmethod
    def parse_xml_result(xml_content: str) -> List[Dict]:
        """แยกผลลัพธ์ที่ใช้ XML tags <page id="X">"""
        pages = []
        page_matches = re.findall(
            r'<page\s+id=[\"\'](\d+)[\"\']>(.*?)</page>',
            xml_content, re.DOTALL | re.IGNORECASE
        )
        for page_num_str, content in page_matches:
            content = ThaiTextUtils.normalize_unicode(content.strip())
            pages.append({
                'page_num': int(page_num_str),
                'cleaned_text': content
            })
        return pages

    @staticmethod
    def extract_metadata(content: str) -> Dict:
        """แยก metadata จาก header ของไฟล์"""
        metadata = {}
        patterns = {
            'book_title': r'### 📘 ชื่อหนังสือ.*?:\s*(.*)',
            'chapter': r'### 🧾 Chapter:\s*(.*)',
            'sub_chapter': r'### 🔖 Sub-Chapter:\s*(.*)',
            'format': r'### 📂 Format:\s*(.*)',
            'purpose': r'### 🧠 Purpose:\s*(.*)'
        }
        for key, pattern in patterns.items():
            match = re.search(pattern, content)
            if match:
                metadata[key] = match.group(1).strip()
        return metadata

print("✅ Multi-page Parser ready")

✅ Multi-page Parser ready


In [6]:
# ============================================
# 📌 Block 6: Enhanced LLM Client with Metadata Analysis
# ============================================
class LLMClient:
    """Universal LLM Client with enhanced error handling และ metadata analysis"""

    def __init__(self):
        self.model = Config.MODEL
        self.client = None
        self.total_tokens = 0
        self.total_cost = 0
        self.usage_logger = UsageLogger()
        if 'gpt' in self.model:
            self._init_openai()
        elif 'claude' in self.model:
            self._init_anthropic()

    def _init_openai(self):
        """Initialize OpenAI client (SDK v1.0+)"""
        try:
            from openai import OpenAI
            if not Config.OPENAI_API_KEY:
                raise ValueError("❌ ไม่พบ OpenAI API key! กรุณาตรวจสอบไฟล์ key")
            self.client = OpenAI(api_key=Config.OPENAI_API_KEY)
            print(f"✅ OpenAI client ready (Model: {self.model})")
        except ImportError:
            print("❌ ต้องติดตั้ง: pip install --upgrade openai>=1.0.0")
            raise

    def _init_anthropic(self):
        """Initialize Anthropic client"""
        try:
            import anthropic
            api_key = Config.ANTHROPIC_API_KEY or os.getenv('ANTHROPIC_API_KEY')
            if not api_key:
                raise ValueError("❌ ไม่พบ Anthropic API key!")
            self.client = anthropic.Anthropic(api_key=api_key)
            print(f"✅ Anthropic client ready (Model: {self.model})")
        except ImportError:
            print("❌ ต้องติดตั้ง: pip install anthropic")
            raise

    def _create_multipage_xml_prompt(self, pages: List[Dict]) -> str:
        """สร้าง prompt สำหรับแก้ไข OCR หลายหน้า"""
        page_texts = [f"--- หน้าที่ {p['page_num']} ---\n{p['raw_text']}" for p in pages]
        combined_text = "\n\n".join(page_texts)
        return f"""แก้ไขข้อความ OCR จากนิยายภาษาไทยหลายหน้าต่อไปนี้

🚨 กฎการแก้ไข (เข้มงวด):
1. แก้เฉพาะ typo และการสะกดผิด
2. แก้คำที่ขาดหาย/แตกหัก (เช่น "มา กำลัง" → "มากำลัง")
3. ลบตัวอักษรเดี่ยวที่ไม่มีความหมาย (เช่น ตัว ก อ ย เดี่ยว ๆ)
4. รักษารูปแบบบทสนทนา (คำพูดใน "...")
5. ❌ ห้ามเพิ่มประโยคใหม่
6. ❌ ห้ามลบประโยค
7. ❌ ห้ามเปลี่ยนความหมาย
8. ❌ ห้ามสรุป/เรียงใหม่
9. ❌ ห้ามแก้ชื่อตัวละคร/สถานที่
10. รักษาลำดับคำเดิมให้มากที่สุด

📋 รูปแบบผลลัพธ์ (บังคับ):
ส่งคืนเฉพาะข้อความในรูปแบบนี้เท่านั้น:

<page num="1">
ข้อความที่แก้แล้วหน้า 1
</page>
<page num="2">
ข้อความที่แก้แล้วหน้า 2
</page>

ห้ามใส่คำอธิบาย คำนำ หรือข้อความอื่นใด

📝 ข้อความ OCR:
{combined_text}

🔄 ผลลัพธ์:"""

    def _create_single_page_prompt(self, text: str) -> str:
        """สร้าง prompt สำหรับแก้ไข OCR หน้าเดียว"""
        return f"""แก้ไขข้อความ OCR จากนิยายภาษาไทยต่อไปนี้

🚨 กฎการแก้ไข (เข้มงวด):
1. แก้เฉพาะ typo และการสะกดผิด
2. แก้คำที่ขาดหาย/แตกหัก (เช่น "มา กำลัง" → "มากำลัง")
3. ลบตัวอักษรเดี่ยวที่ไม่มีความหมาย
4. รักษารูปแบบบทสนทนา (คำพูดใน "...")
5. ❌ ห้ามเพิ่มประโยคใหม่
6. ❌ ห้ามลบประโยค
7. ❌ ห้ามเปลี่ยนความหมาย
8. ❌ ห้ามแก้ชื่อตัวละคร/สถานที่

ส่งคืนเฉพาะข้อความที่แก้แล้ว ไม่ต้องมีคำอธิบาย

ข้อความ OCR:
{text}

ข้อความที่แก้แล้ว:"""

    def _create_metadata_analysis_prompt(self, page_text: str, page_num: int, book_info: Dict) -> str:
        """สร้าง prompt สำหรับวิเคราะห์ metadata ของแต่ละหน้า"""
        return f"""วิเคราะห์หน้านิยายภาษาไทยต่อไปนี้และสกัด metadata ตามรูปแบบที่กำหนด

📚 ข้อมูลหนังสือ:
- ชื่อเรื่อง: {book_info.get('book_title', 'ไม่ระบุ')}
- บท: {book_info.get('chapter', 'ไม่ระบุ')}
- หน้า: {page_num}

📝 ข้อความในหน้า:
{page_text[:2500]}  # จำกัด 2500 ตัวอักษรเพื่อประหยัด token

🎯 วิเคราะห์และตอบเป็น JSON ดังนี้ (ห้ามใส่ comment หรือคำอธิบาย):
{{
  "tone": ["อารมณ์หลักของหน้านี้ เช่น dramatic, tense, romantic"],
  "tags": ["แท็กเฉพาะหน้านี้ ถ้าไม่ชัดเจนให้ใช้ general"],
  "characters": ["ชื่อตัวละครที่ปรากฏในหน้านี้เท่านั้น"],
  "places": ["สถานที่ในหน้านี้"],
  "objects": ["วัตถุสำคัญในหน้านี้"],
  "dialogue_pairs": ตัวเลขจำนวนคู่บทสนทนา,
  "char_count": จำนวนตัวอักษรโดยประมาณ,
  "word_count": จำนวนคำโดยประมาณ,
  "paragraph_count": จำนวนย่อหน้า,
  "style_notes": "สำนวนการเขียนหรือจังหวะของหน้านี้ (1-2 บรรทัด)",
  "summary": "สรุป 1 บรรทัดไม่เกิน 25 คำ",
  "anomalies": "สิ่งผิดปกติถ้ามี หรือ null",
  "confidence": 0.85
}}

ตอบเฉพาะ JSON เท่านั้น ไม่ต้องมีคำอธิบาย:"""

    def _call_api_with_retry(self, prompt: str, filename: str, retry_count: int) -> Dict:
        """เรียก API พร้อม retry mechanism"""
        for attempt in range(Config.MAX_RETRIES + 1):
            try:
                if 'gpt' in self.model:
                    return self._call_openai(prompt)
                else:
                    return self._call_anthropic(prompt)
            except Exception as e:
                error_type = type(e).__name__
                print(f"   ❌ API error (attempt {attempt + 1}): {error_type}")
                if attempt < Config.MAX_RETRIES:
                    base_delay = 2 ** attempt
                    jitter = random.uniform(0.5, 1.5)
                    delay = base_delay * jitter
                    print(f"   ⏱ Retrying in {delay:.1f}s...")
                    time.sleep(delay)
                else:
                    raise Exception(f"API failed after {Config.MAX_RETRIES + 1} attempts: {e}")

    def _call_openai(self, prompt: str) -> Dict:
        """เรียก OpenAI API"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "คุณคือผู้เชี่ยวชาญวิเคราะห์นิยายภาษาไทย ตอบตามรูปแบบที่กำหนดอย่างเคร่งครัด"},
                {"role": "user", "content": prompt}
            ],
            temperature=Config.TEMPERATURE,
            max_tokens=Config.MAX_TOKENS
        )
        cleaned_text = response.choices[0].message.content
        total_tokens = response.usage.total_tokens
        input_tokens = getattr(response.usage, 'prompt_tokens', total_tokens // 2)
        output_tokens = getattr(response.usage, 'completion_tokens', total_tokens // 2)
        prices = Config.PRICE_PER_1K_TOKENS.get(self.model, {'input': 0.0005, 'output': 0.0015})
        input_cost = (input_tokens / 1000) * prices['input']
        output_cost = (output_tokens / 1000) * prices['output']
        total_cost = input_cost + output_cost
        return {
            'cleaned_text': cleaned_text,
            'tokens_used': total_tokens,
            'input_tokens': input_tokens,
            'output_tokens': output_tokens,
            'cost': total_cost,
            'model': self.model
        }

    def _call_anthropic(self, prompt: str) -> Dict:
        """เรียก Anthropic API"""
        response = self.client.messages.create(
            model=self.model,
            max_tokens=Config.MAX_TOKENS,
            temperature=Config.TEMPERATURE,
            messages=[{"role": "user", "content": prompt}]
        )
        cleaned_text = response.content[0].text
        input_tokens = getattr(response.usage, 'input_tokens', 0)
        output_tokens = getattr(response.usage, 'output_tokens', 0)
        total_tokens = input_tokens + output_tokens
        prices = Config.PRICE_PER_1K_TOKENS.get(self.model, {'input': 0.00025, 'output': 0.00125})
        input_cost = (input_tokens / 1000) * prices['input']
        output_cost = (output_tokens / 1000) * prices['output']
        total_cost = input_cost + output_cost
        return {
            'cleaned_text': cleaned_text,
            'tokens_used': total_tokens,
            'input_tokens': input_tokens,
            'output_tokens': output_tokens,
            'cost': total_cost,
            'model': self.model
        }

    def analyze_page_metadata(self, page_text: str, page_num: int, book_info: Dict, filename: str = "") -> Dict:
        """
        วิเคราะห์ metadata ของหน้าเดียว

        Args:
            page_text: ข้อความในหน้าที่จะวิเคราะห์
            page_num: หมายเลขหน้า
            book_info: ข้อมูลหนังสือ (title, chapter, etc.)
            filename: ชื่อไฟล์สำหรับ logging

        Returns:
            Dict ที่มี metadata ของหน้า
        """
        start_time = time.time()

        # สร้าง prompt
        prompt = self._create_metadata_analysis_prompt(page_text, page_num, book_info)

        # เรียก API
        result = self._call_api_with_retry(prompt, f"{filename}_meta_p{page_num}", 0)

        # พยายาม parse JSON จาก response
        try:
            import json
            # ลบ markdown code block ถ้ามี
            response_text = result['cleaned_text']
            response_text = response_text.replace('```json\n', '').replace('\n```', '')
            response_text = response_text.replace('```', '')

            metadata = json.loads(response_text)

            # ตรวจสอบ required fields
            required_fields = ['tone', 'tags', 'characters', 'places', 'objects',
                             'dialogue_pairs', 'style_notes', 'summary', 'confidence']
            for field in required_fields:
                if field not in metadata:
                    metadata[field] = [] if field in ['tone', 'tags', 'characters', 'places', 'objects'] else ""

        except json.JSONDecodeError as e:
            print(f"   ⚠️ Failed to parse metadata for page {page_num}: {e}")
            # Return default metadata
            metadata = {
                'tone': ['unknown'],
                'tags': ['general'],
                'characters': [],
                'places': [],
                'objects': [],
                'dialogue_pairs': 0,
                'char_count': len(page_text),
                'word_count': len(page_text.split()),
                'paragraph_count': page_text.count('\n\n') + 1,
                'style_notes': 'ไม่สามารถวิเคราะห์ได้',
                'summary': 'ไม่สามารถสรุปได้',
                'anomalies': 'JSON parsing failed',
                'confidence': 0.0
            }

        # เพิ่มข้อมูลการประมวลผล
        processing_time = time.time() - start_time
        metadata['processing_time'] = processing_time
        metadata['tokens_used'] = result.get('tokens_used', 0)
        metadata['cost'] = result.get('cost', 0)

        # Log usage
        self.usage_logger.log_usage(
            original_filename=filename,
            clean_filename=f"{filename}_metadata_p{page_num}",
            pages_count=1,
            model=self.model,
            input_tokens=result.get('input_tokens', 0),
            output_tokens=result.get('output_tokens', 0),
            cost_usd=result.get('cost', 0),
            processing_time=processing_time,
            retry_count=0,
            validation_status='META_ANALYSIS'
        )

        self.total_tokens += result.get('tokens_used', 0)
        self.total_cost += result.get('cost', 0)

        return metadata

    def clean_multipage_ocr(self, pages: List[Dict], filename: str = "") -> Dict:
        """แก้ไข OCR หลายหน้าพร้อมกัน"""
        start_time = time.time()
        retry_count = 0
        prompt = self._create_multipage_xml_prompt(pages)
        result = self._call_api_with_retry(prompt, filename, retry_count)
        cleaned_pages = self._parse_xml_result(result['cleaned_text'], pages)
        processing_time = time.time() - start_time
        self.total_tokens += result['tokens_used']
        self.total_cost += result['cost']
        self.usage_logger.log_usage(
            original_filename=filename,
            clean_filename=f"{filename}_processed",
            pages_count=len(pages),
            model=self.model,
            input_tokens=result.get('input_tokens', 0),
            output_tokens=result.get('output_tokens', 0),
            cost_usd=result['cost'],
            processing_time=processing_time,
            retry_count=retry_count
        )
        return {
            'cleaned_pages': cleaned_pages,
            'tokens_used': result['tokens_used'],
            'cost': result['cost'],
            'processing_time': processing_time,
            'input_tokens': result.get('input_tokens', 0),
            'output_tokens': result.get('output_tokens', 0)
        }

    def clean_ocr_text(self, text: str, filename: str = "", page_num: int = 1) -> Dict:
        """แก้ไข OCR หน้าเดียว"""
        start_time = time.time()
        retry_count = 0
        prompt = self._create_single_page_prompt(text)
        result = self._call_api_with_retry(prompt, filename, retry_count)
        processing_time = time.time() - start_time
        self.total_tokens += result['tokens_used']
        self.total_cost += result['cost']
        self.usage_logger.log_usage(
            original_filename=filename,
            clean_filename=f"{filename}_processed_p{page_num}",
            pages_count=1,
            model=self.model,
            input_tokens=result.get('input_tokens', 0),
            output_tokens=result.get('output_tokens', 0),
            cost_usd=result['cost'],
            processing_time=processing_time,
            retry_count=retry_count
        )
        return result

    def _parse_xml_result(self, cleaned_text: str, original_pages: List[Dict]) -> List[Dict]:
        """แยกผลลัพธ์จาก XML tags"""
        cleaned_pages = []
        xml_pages = MultiPageParser.parse_xml_result(cleaned_text)
        xml_dict = {p['page_num']: p['cleaned_text'] for p in xml_pages}
        for original_page in original_pages:
            page_num = original_page['page_num']
            cleaned_content = xml_dict.get(page_num, "")
            if not cleaned_content:
                print(f"   ⚠️ Warning: ไม่พบผลลัพธ์สำหรับหน้า {page_num}")
                cleaned_content = original_page['raw_text']
            cleaned_pages.append({
                'page_num': page_num,
                'raw_text': original_page['raw_text'],
                'cleaned_text': cleaned_content
            })
        return cleaned_pages

    def generate_text(self, prompt: str, filename: str = "", system: str = "You are a helpful assistant.") -> Dict:
        """Generate text ทั่วไป"""
        start_time = time.time()
        retry_count = 0
        for attempt in range(Config.MAX_RETRIES + 1):
            try:
                if 'gpt' in self.model:
                    response = self.client.chat.completions.create(
                        model=self.model,
                        messages=[{"role": "system", "content": system},
                                  {"role": "user", "content": prompt}],
                        temperature=Config.TEMPERATURE,
                        max_tokens=Config.MAX_TOKENS
                    )
                    result_text = response.choices[0].message.content
                    total_tokens = response.usage.total_tokens
                    input_tokens = getattr(response.usage, 'prompt_tokens', total_tokens // 2)
                    output_tokens = getattr(response.usage, 'completion_tokens', total_tokens // 2)
                else:
                    response = self.client.messages.create(
                        model=self.model,
                        max_tokens=Config.MAX_TOKENS,
                        temperature=Config.TEMPERATURE,
                        messages=[{"role": "user", "content": prompt}]
                    )
                    result_text = response.content[0].text
                    input_tokens = getattr(response.usage, 'input_tokens', 0)
                    output_tokens = getattr(response.usage, 'output_tokens', 0)
                    total_tokens = input_tokens + output_tokens

                prices = Config.PRICE_PER_1K_TOKENS.get(self.model, {'input': 0.0005, 'output': 0.0015})
                input_cost = (input_tokens / 1000) * prices['input']
                output_cost = (output_tokens / 1000) * prices['output']
                total_cost = input_cost + output_cost

                processing_time = time.time() - start_time
                self.usage_logger.log_usage(
                    original_filename=filename,
                    clean_filename=f"{filename}_generated",
                    pages_count=1,
                    model=self.model,
                    input_tokens=input_tokens,
                    output_tokens=output_tokens,
                    cost_usd=total_cost,
                    processing_time=processing_time,
                    retry_count=retry_count
                )
                self.total_tokens += total_tokens
                self.total_cost += total_cost
                return {
                    'text': result_text,
                    'tokens_used': total_tokens,
                    'input_tokens': input_tokens,
                    'output_tokens': output_tokens,
                    'cost': total_cost,
                    'processing_time': processing_time
                }
            except Exception as e:
                retry_count += 1
                if attempt < Config.MAX_RETRIES:
                    base_delay = 2 ** attempt
                    jitter = random.uniform(0.5, 1.5)
                    delay = base_delay * jitter
                    print(f"   Retry {attempt + 1}/{Config.MAX_RETRIES} in {delay:.1f}s...")
                    time.sleep(delay)
                else:
                    raise Exception(f"Generate text failed: {e}")

    def get_usage_summary(self) -> Dict:
        """สรุปการใช้งาน API"""
        file_summary = self.usage_logger.get_summary()
        return {
            'total_tokens': self.total_tokens,
            'total_cost_usd': self.total_cost,
            'total_cost_thb': self.total_cost * 35,
            'pages_processed': self.total_tokens // 500,
            'session_files': file_summary.get('total_files', 0),
            'session_pages': file_summary.get('total_pages', 0),
            'avg_cost_per_page': file_summary.get('avg_cost_per_page', 0),
            'detailed_stats': file_summary
        }

print("✅ Enhanced LLM Client with Metadata Analysis ready")

✅ Enhanced LLM Client with Metadata Analysis ready


In [7]:
# ============================================
# 📌 Block 7: Enhanced Text Chunker
# ============================================
class EnhancedChunker:
    """แบ่งข้อความด้วย context overlap และ smart boundary detection"""

    @staticmethod
    def smart_chunk_text(text: str, max_chars: int = 2500) -> List[str]:
        """แบ่งข้อความอย่างชาญฉลาดด้วย context overlap"""
        sentence_endings = ['.', '!', '?', '…', '"', '"']
        paragraph_break = '\n\n'

        chunks = []
        current_pos = 0
        half = max_chars // 2

        while current_pos < len(text):
            chunk_end = min(current_pos + max_chars, len(text))
            if chunk_end < len(text):
                para_pos = text.rfind(paragraph_break, current_pos, chunk_end)
                if para_pos > current_pos + half:
                    chunk_end = para_pos + len(paragraph_break)
                else:
                    best_pos = -1
                    for ending in sentence_endings:
                        pos = text.rfind(ending, current_pos + half, chunk_end)
                        if pos > best_pos:
                            best_pos = pos + len(ending)
                    if best_pos > current_pos:
                        chunk_end = best_pos

            chunk = text[current_pos:chunk_end]

            if chunks and Config.CONTEXT_OVERLAP > 0:
                overlap_start = max(0, current_pos - Config.CONTEXT_OVERLAP)
                overlap_text = text[overlap_start:current_pos]
                if overlap_text:
                    chunk = f"[บริบทก่อนหน้า: {overlap_text[-50:]}...]\n\n{chunk}"

            chunks.append(chunk)
            prev_pos = current_pos
            current_pos = chunk_end
            if current_pos == prev_pos and current_pos < len(text):
                current_pos += 1

        return chunks

    @staticmethod
    def merge_chunks_with_dedup(chunks: List[str]) -> str:
        """รวม chunks โดยลบ overlap ซ้ำ"""
        if not chunks:
            return ""
        if len(chunks) == 1:
            return chunks[0]

        result = chunks[0]
        for i in range(1, len(chunks)):
            chunk = chunks[i]
            if chunk.startswith('[บริบทก่อนหน้า:'):
                cut = chunk.find(']\n\n')
                if cut != -1:
                    chunk = chunk[cut + 4:]
            result += chunk
        return result

print("✅ Enhanced Chunker ready")

✅ Enhanced Chunker ready


In [8]:
# ============================================
# 📌 Block 8: Thai-specific Validator
# ============================================
class ThaiValidator:
    """ตรวจสอบคุณภาพสำหรับภาษาไทยเฉพาะ"""

    @staticmethod
    def validate_thai_text(raw_text: str, cleaned_text: str, filename: str = "") -> Dict:
        """ตรวจสอบแบบเฉพาะภาษาไทย"""
        issues, warnings = [], []

        # 1) ความยาว
        len_ratio = len(cleaned_text) / len(raw_text) if raw_text else 0
        len_change = (len(cleaned_text) - len(raw_text)) / len(raw_text) * 100 if raw_text else 0
        if len_ratio < 0.7:
            issues.append(f"⚠️ ข้อความสั้นลงมาก {abs(len_change):.1f}% (อาจมีการลบเนื้อหา)")
        elif len_ratio > 1.3:
            issues.append(f"⚠️ ข้อความยาวขึ้นมาก {len_change:.1f}% (อาจมีการเพิ่มเนื้อหา)")
        elif len_ratio < 0.85:
            warnings.append(f"📝 ข้อความสั้นลง {abs(len_change):.1f}%")
        elif len_ratio > 1.15:
            warnings.append(f"📝 ข้อความยาวขึ้น {len_change:.1f}%")

        # 2) อัญประกาศ
        raw_quotes = ThaiTextUtils.count_thai_quotes(raw_text)
        clean_quotes = ThaiTextUtils.count_thai_quotes(cleaned_text)
        total_raw = sum(raw_quotes.values())
        total_clean = sum(clean_quotes.values())
        if abs(total_raw - total_clean) > 3:
            issues.append(f"⚠️ อัญประกาศเปลี่ยนมาก ({total_raw} → {total_clean})")
        elif abs(total_raw - total_clean) > 1:
            warnings.append(f"📝 อัญประกาศเปลี่ยน ({total_raw} → {total_clean})")

        # 3) ไม้ยมก
        raw_yamok = raw_text.count('ๆ')
        clean_yamok = cleaned_text.count('ๆ')
        if abs(raw_yamok - clean_yamok) > 2:
            warnings.append(f"📝 ไม้ยมก (ๆ) เปลี่ยน ({raw_yamok} → {clean_yamok})")

        # 4) ตัวเลข
        raw_numbers = len(re.findall(r'\d+', raw_text))
        clean_numbers = len(re.findall(r'\d+', cleaned_text))
        if abs(raw_numbers - clean_numbers) > 3:
            issues.append(f"⚠️ ตัวเลขเปลี่ยนมาก ({raw_numbers} → {clean_numbers})")
        elif abs(raw_numbers - clean_numbers) > 1:
            warnings.append(f"📝 ตัวเลขเปลี่ยน ({raw_numbers} → {clean_numbers})")

        # 5) คำแตกหัก
        broken_words_raw = ThaiTextUtils.detect_word_breaks(raw_text)
        broken_words_clean = ThaiTextUtils.detect_word_breaks(cleaned_text)
        if len(broken_words_clean) > len(broken_words_raw) * 0.5:
            warnings.append(f"📝 ยังมีคำแตกหัก: {broken_words_clean[:3]}")

        # Score & Status
        score = 1.0 - 0.3*len(issues) - 0.1*len(warnings)
        score = max(0, min(1, score))
        status = 'PASS'
        if issues:
            status = 'FAIL'
        elif warnings:
            status = 'WARNING'

        return {
            'filename': filename,
            'status': status,
            'score': score,
            'issues': issues,
            'warnings': warnings,
            'stats': {
                'length_change': f"{len_change:+.1f}%",
                'quotes_change': f"{total_raw} → {total_clean}",
                'yamok_change': f"{raw_yamok} → {clean_yamok}",
                'numbers_change': f"{raw_numbers} → {clean_numbers}",
                'broken_words_remaining': len(broken_words_clean)
            },
            'timestamp': datetime.now().isoformat()
        }

print("✅ Thai Validator ready")

✅ Thai Validator ready


In [9]:
# ============================================
# 📌 Block 9: Enhanced OCR Processor with Metadata
# ============================================
class OCRProcessor:
    """Main processor with metadata analysis สำหรับแต่ละหน้า"""

    def __init__(self, analyze_metadata: bool = True):
        """
        Args:
            analyze_metadata: ถ้า True จะวิเคราะห์ metadata ของแต่ละหน้า (มีค่าใช้จ่ายเพิ่ม)
        """
        self.llm = LLMClient()
        self.chunker = EnhancedChunker()
        self.validator = ThaiValidator()
        self.analyze_metadata = analyze_metadata
        self.stats = {
            'processed': 0,
            'failed': 0,
            'validation_pass': 0,
            'validation_warning': 0,
            'validation_fail': 0,
            'multipage_files': 0,
            'single_page_files': 0,
            'metadata_analyzed': 0
        }
        self.training_pairs = []
        self.metadata_collection = []  # เก็บ metadata ทั้งหมดสำหรับ analysis

    # ---------- Top-level ----------
    def process_file(self, file_path: Path) -> Dict:
        """Process ไฟล์ OCR พร้อม metadata analysis"""
        print(f"\n📄 Processing: {file_path.name}")
        if self.analyze_metadata:
            print("   🔍 Metadata analysis: ENABLED")
        try:
            raw_content = file_path.read_text(encoding='utf-8')
            if "--- PAGE:" in raw_content:
                return self._process_multipage_file(file_path, raw_content)
            else:
                normalized_content = ThaiTextUtils.normalize_unicode(raw_content)
                return self._process_single_page_file(file_path, normalized_content)
        except Exception as e:
            print(f"   ❌ Error: {e}")
            self.stats['failed'] += 1
            return {'success': False, 'error': str(e)}

    # ---------- Multi-page with Metadata ----------
    def _process_multipage_file(self, file_path: Path, content: str) -> Dict:
        """Process multi-page file พร้อม metadata analysis"""
        print("   📖 Multi-page file detected")
        metadata = MultiPageParser.extract_metadata(content)
        pages = MultiPageParser.parse_multipage_file(content)
        print(f"   📊 Found {len(pages)} pages")
        if metadata:
            print(f"   📘 Book: {metadata.get('book_title', 'Unknown')}")
            print(f"   🧾 Chapter: {metadata.get('chapter', 'Unknown')}")

        total_chars = sum(len(p['raw_text']) for p in pages)
        estimated_tokens = total_chars * 0.75
        print(f"   📏 Total content: {total_chars:,} chars (~{estimated_tokens:,.0f} tokens)")

        # Process cleaning
        if len(pages) <= Config.MAX_PAGES_PER_BATCH:
            print(f"   🚀 Processing all {len(pages)} pages in one batch...")
            result = self.llm.clean_multipage_ocr(pages, file_path.name)
            cleaned_pages = result['cleaned_pages']
            total_cost = result['cost']
            total_tokens = result['tokens_used']
            processing_time = result.get('processing_time', 0)
        else:
            print(f"   📦 Processing in batches (max {Config.MAX_PAGES_PER_BATCH} pages/batch)...")
            cleaned_pages, total_cost, total_tokens, processing_time = [], 0, 0, 0
            for i in range(0, len(pages), Config.MAX_PAGES_PER_BATCH):
                batch = pages[i:i + Config.MAX_PAGES_PER_BATCH]
                batch_num = i // Config.MAX_PAGES_PER_BATCH + 1
                print(f"      Batch {batch_num}: Pages {batch[0]['page_num']}-{batch[-1]['page_num']}")
                result = self.llm.clean_multipage_ocr(batch, f"{file_path.name}_batch_{batch_num}")
                cleaned_pages.extend(result['cleaned_pages'])
                total_cost += result['cost']
                total_tokens += result['tokens_used']
                processing_time += result.get('processing_time', 0)
                if i + Config.MAX_PAGES_PER_BATCH < len(pages):
                    time.sleep(2)

        # Analyze metadata for each page (ถ้าเปิดใช้งาน)
        page_metadata_dict = {}
        metadata_cost = 0
        metadata_tokens = 0

        if self.analyze_metadata:
            print("   🔍 Analyzing metadata for each page...")
            for page in cleaned_pages:
                page_num = page['page_num']
                print(f"      Analyzing page {page_num}...")

                # วิเคราะห์ metadata จาก cleaned text
                page_meta = self.llm.analyze_page_metadata(
                    page_text=page['cleaned_text'],
                    page_num=page_num,
                    book_info=metadata,
                    filename=file_path.name
                )

                page_metadata_dict[page_num] = page_meta
                metadata_cost += page_meta.get('cost', 0)
                metadata_tokens += page_meta.get('tokens_used', 0)
                self.stats['metadata_analyzed'] += 1

                # เก็บไว้สำหรับ analysis ภายหลัง
                self.metadata_collection.append({
                    'file': file_path.name,
                    'page': page_num,
                    'metadata': page_meta
                })

                # หน่วงเวลาเล็กน้อยเพื่อไม่ให้ API rate limit
                time.sleep(0.5)

            print(f"   ✅ Metadata analysis complete: {metadata_tokens:,} tokens, ${metadata_cost:.4f}")
            total_cost += metadata_cost
            total_tokens += metadata_tokens

        # Validation
        validation_results = []
        for page in cleaned_pages:
            if page.get('raw_text') and page.get('cleaned_text'):
                val = self.validator.validate_thai_text(
                    page['raw_text'], page['cleaned_text'], f"{file_path.name}_p{page['page_num']}"
                )
                validation_results.append(val)

        validation_summary = self._summarize_validation(validation_results)

        # ---------- สร้างไฟล์ output ----------
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        clean_filename = f"{file_path.stem}_clean_{timestamp}.txt"
        clean_path = Path(Config.CLEANED_DIR) / clean_filename

        output_content = []
        if metadata:
            if 'book_title' in metadata:
                output_content.append(f"### 📘 ชื่อหนังสือ (Book Title): {metadata['book_title']}")
            if 'chapter' in metadata:
                output_content.append(f"### 🧾 Chapter: {metadata['chapter']}")
            if 'sub_chapter' in metadata:
                output_content.append(f"### 🔖 Sub-Chapter: {metadata['sub_chapter']}")
            output_content.append(f"### 📂 Format: CLEANED (v3.0)")
            if 'purpose' in metadata:
                output_content.append(f"### 🧠 Purpose: {metadata['purpose']}")
        output_content.append(f"### ⚙️ Processing: {datetime.now().isoformat()}")
        output_content.append(f"### 📊 Stats: {len(cleaned_pages)} pages, {total_tokens:,} tokens, ${total_cost:.4f}")
        output_content.append(f"### ✅ Validation: {validation_summary}")
        output_content.append("")

        # เขียนแต่ละหน้าพร้อม metadata
        for page in cleaned_pages:
            page_num = page['page_num']
            output_content.append(f"--- PAGE: {page_num} ---")

            # เพิ่ม metadata ถ้ามี
            if page_num in page_metadata_dict:
                meta = page_metadata_dict[page_num]
                output_content.append(f"### 📘 Book: {metadata.get('book_title', '')}")
                output_content.append(f"### 🧾 Chapter: {metadata.get('chapter', '')}")
                output_content.append(f"### 📄 Page: {page_num}")
                output_content.append(f"🗣️ Tone: {', '.join(meta.get('tone', ['unknown']))}")
                output_content.append(f"🏷️ Tags: {', '.join(meta.get('tags', ['general']))}")
                output_content.append(f"👥 Characters: {', '.join(meta.get('characters', []))}")
                output_content.append(f"📍 Places: {', '.join(meta.get('places', []))}")
                output_content.append(f"🔸 Objects: {', '.join(meta.get('objects', []))}")
                output_content.append(f"💬 Dialogue Pairs: {meta.get('dialogue_pairs', 0)}")
                output_content.append(
                    f"📊 Stats: Chars≈{meta.get('char_count', 0)} | Words≈{meta.get('word_count', 0)} | Paragraphs≈{meta.get('paragraph_count', 0)}"
                )
                output_content.append(f"✏️ Style Notes: {meta.get('style_notes', '')}")
                output_content.append(f"📝 One-line Summary: {meta.get('summary', '')}")
                if meta.get('anomalies'):
                    output_content.append(f"⚠️ Anomalies: {meta.get('anomalies')}")
                output_content.append(f"Confidence: {meta.get('confidence', 0.0)}")

            output_content.append("--- RAW ---")
            output_content.append(page['raw_text'])
            output_content.append("--- CLEANED ---")
            output_content.append(page['cleaned_text'])
            output_content.append("")

        clean_path.write_text('\n'.join(output_content), encoding='utf-8')

        # เก็บ training pairs (เฉพาะที่ validation ไม่ FAIL)
        for page, val in zip(cleaned_pages, validation_results):
            if page.get('raw_text') and page.get('cleaned_text') and val['status'] != 'FAIL':
                self.training_pairs.append({
                    'input': page['raw_text'][:1000],
                    'output': page['cleaned_text'][:1000],
                    'source': f"{file_path.name}_page_{page['page_num']}",
                    'timestamp': timestamp,
                    'validation_score': val['score'],
                    'metadata': metadata,
                    'page_metadata': page_metadata_dict.get(page['page_num'], {})
                })

        # Update stats
        self.stats['processed'] += 1
        self.stats['multipage_files'] += 1
        for val in validation_results:
            self.stats[f"validation_{val['status'].lower()}"] += 1

        print(f"   ✅ Saved: {clean_filename}")
        print(f"   📄 Pages: {len(cleaned_pages)}")
        print(f"   🔤 Tokens: {total_tokens:,}")
        print(f"   ⏱ Time: {processing_time:.1f}s")
        print(f"   💰 Cost: ${total_cost:.4f} (~{total_cost*35:.2f} บาท)")
        print(f"   📋 Validation: {validation_summary}")
        if self.analyze_metadata:
            print(f"   🔍 Metadata analyzed: {len(page_metadata_dict)} pages")

        return {
            'success': True,
            'cleaned_path': str(clean_path),
            'pages_count': len(cleaned_pages),
            'tokens': total_tokens,
            'cost': total_cost,
            'processing_time': processing_time,
            'validation_summary': validation_summary,
            'metadata': metadata,
            'page_metadata': page_metadata_dict if self.analyze_metadata else {}
        }

    # ---------- Single-page with Metadata ----------
    def _process_single_page_file(self, file_path: Path, content: str) -> Dict:
        """Process single page file พร้อม metadata analysis"""
        print("   📄 Single page file")

        # Clean OCR
        if len(content) > 3000:
            chunks = self.chunker.smart_chunk_text(content, 2500)
            cleaned_chunks, total_cost, total_tokens, processing_time = [], 0, 0, 0
            print(f"   📦 Split into {len(chunks)} smart chunks")
            for i, chunk in enumerate(chunks, 1):
                print(f"      Chunk {i}/{len(chunks)}...")
                result = self.llm.clean_ocr_text(chunk, filename=f"{file_path.name}_chunk_{i}")
                cleaned_chunks.append(result['cleaned_text'])
                total_cost += result.get('cost', 0)
                total_tokens += result.get('tokens_used', 0)
                processing_time += result.get('processing_time', 0)
                time.sleep(1)
            cleaned_text = self.chunker.merge_chunks_with_dedup(cleaned_chunks)
        else:
            result = self.llm.clean_ocr_text(content, filename=file_path.name)
            cleaned_text = result['cleaned_text']
            total_cost = result['cost']
            total_tokens = result['tokens_used']
            processing_time = result.get('processing_time', 0)

        # Analyze metadata ถ้าเปิดใช้งาน
        page_metadata = {}
        if self.analyze_metadata:
            print("   🔍 Analyzing metadata...")
            page_metadata = self.llm.analyze_page_metadata(
                page_text=cleaned_text,
                page_num=1,
                book_info={'book_title': file_path.stem},
                filename=file_path.name
            )
            total_cost += page_metadata.get('cost', 0)
            total_tokens += page_metadata.get('tokens_used', 0)
            self.stats['metadata_analyzed'] += 1

        # Validation
        validation_result = self.validator.validate_thai_text(content, cleaned_text, file_path.name)

        # Create output file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        clean_filename = (f"WARNING_{file_path.stem}_clean_{timestamp}.txt"
                          if validation_result['status'] == 'FAIL'
                          else f"{file_path.stem}_clean_{timestamp}.txt")
        clean_path = Path(Config.CLEANED_DIR) / clean_filename

        # Build output content
        output_lines = []
        # Header
        output_lines.append(f"### 📘 File: {file_path.name}")
        output_lines.append(f"### 📂 Format: CLEANED (v3.0)")
        output_lines.append(f"### ⚙️ Processing: {datetime.now().isoformat()}")
        output_lines.append(f"### 📊 Stats: {total_tokens:,} tokens, ${total_cost:.4f}")
        output_lines.append(f"### ✅ Validation: {validation_result['status']} (score: {validation_result['score']:.2f})")
        output_lines.append("")

        # Metadata ถ้ามี
        if page_metadata:
            output_lines.append("--- METADATA ---")
            output_lines.append(f"🗣️ Tone: {', '.join(page_metadata.get('tone', ['unknown']))}")
            output_lines.append(f"🏷️ Tags: {', '.join(page_metadata.get('tags', ['general']))}")
            output_lines.append(f"👥 Characters: {', '.join(page_metadata.get('characters', []))}")
            output_lines.append(f"📍 Places: {', '.join(page_metadata.get('places', []))}")
            output_lines.append(f"🔸 Objects: {', '.join(page_metadata.get('objects', []))}")
            output_lines.append(f"💬 Dialogue Pairs: {page_metadata.get('dialogue_pairs', 0)}")
            output_lines.append(
                f"📊 Stats: Chars≈{page_metadata.get('char_count', 0)} | Words≈{page_metadata.get('word_count', 0)} | Paragraphs≈{page_metadata.get('paragraph_count', 0)}"
            )
            output_lines.append(f"✏️ Style Notes: {page_metadata.get('style_notes', '')}")
            output_lines.append(f"📝 One-line Summary: {page_metadata.get('summary', '')}")
            if page_metadata.get('anomalies'):
                output_lines.append(f"⚠️ Anomalies: {page_metadata.get('anomalies')}")
            output_lines.append(f"Confidence: {page_metadata.get('confidence', 0.0)}")
            output_lines.append("")

        # Content
        output_lines.append("--- CLEANED ---")
        output_lines.append(cleaned_text)

        clean_path.write_text('\n'.join(output_lines), encoding='utf-8')

        # Save training pair
        if validation_result['status'] != 'FAIL':
            self.training_pairs.append({
                'input': content[:1000],
                'output': cleaned_text[:1000],
                'source': file_path.name,
                'timestamp': timestamp,
                'validation_score': validation_result['score'],
                'metadata': page_metadata if self.analyze_metadata else {}
            })

        # Update stats
        self.stats['processed'] += 1
        self.stats['single_page_files'] += 1
        self.stats[f"validation_{validation_result['status'].lower()}"] += 1

        print(f"   ✅ Saved: {clean_filename}")
        print(f"   📋 Validation: {validation_result['status']} (score: {validation_result['score']:.2f})")
        print(f"   💰 Cost: ${total_cost:.4f}")
        if self.analyze_metadata:
            print(f"   🔍 Metadata analyzed: 1 page")

        return {
            'success': True,
            'cleaned_path': str(clean_path),
            'tokens': total_tokens,
            'cost': total_cost,
            'processing_time': processing_time,
            'validation_result': validation_result,
            'metadata': page_metadata if self.analyze_metadata else {}
        }

    # ---------- Helper Methods ----------
    def _summarize_validation(self, validation_results: List[Dict]) -> str:
        """สรุปผลการ validation"""
        if not validation_results:
            return "No validation data"
        pass_count = sum(1 for v in validation_results if v['status'] == 'PASS')
        warning_count = sum(1 for v in validation_results if v['status'] == 'WARNING')
        fail_count = sum(1 for v in validation_results if v['status'] == 'FAIL')
        avg_score = sum(v['score'] for v in validation_results) / len(validation_results)
        return f"✅{pass_count} ⚠️{warning_count} ❌{fail_count} (avg: {avg_score:.2f})"

    # ---------- Batch Processing ----------
    def process_batch(self, file_pattern: str = "*.txt", limit: int = None, analyze_metadata: bool = None) -> Dict:
        """
        Process หลายไฟล์จาก RAW_OCR_DIR

        Args:
            file_pattern: รูปแบบไฟล์ที่จะประมวลผล
            limit: จำนวนไฟล์สูงสุด
            analyze_metadata: override การตั้งค่า metadata analysis
        """
        if analyze_metadata is not None:
            self.analyze_metadata = analyze_metadata

        raw_dir = Path(Config.RAW_OCR_DIR)
        files = sorted(list(raw_dir.glob(file_pattern)))
        if limit:
            files = files[:limit]

        print(f"\n🧺 Batch: {len(files)} files found (pattern='{file_pattern}')")
        if self.analyze_metadata:
            print("   🔍 Metadata analysis: ENABLED")
        else:
            print("   ⚡ Metadata analysis: DISABLED (faster, cheaper)")

        results = []
        for i, f in enumerate(files, 1):
            print(f"\n[{i}/{len(files)}] {f.name}")
            res = self.process_file(f)
            results.append({'file': f.name, **res})

        print("\n🏁 Batch done.")
        return {
            'count': len(results),
            'results': results,
            'stats': self.stats
        }

    # ---------- Export Functions ----------
    def export_training_pairs(self, out_name: str = None) -> str:
        """บันทึก training pairs เป็น JSONL"""
        if not self.training_pairs:
            print("⚠️ ยังไม่มี training_pairs ให้บันทึก")
            return ""
        out_dir = Path(Config.TRAINING_PAIRS_DIR)
        out_dir.mkdir(parents=True, exist_ok=True)
        stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_name = out_name or f"training_pairs_{stamp}.jsonl"
        out_path = out_dir / out_name
        with open(out_path, 'w', encoding='utf-8') as f:
            for row in self.training_pairs:
                f.write(json.dumps(row, ensure_ascii=False) + "\n")
        print(f"✅ Exported training pairs: {out_path.name} ({len(self.training_pairs)} rows)")
        return str(out_path)

    def export_metadata_analysis(self, out_name: str = None) -> str:
        """บันทึก metadata analysis เป็น JSON"""
        if not self.metadata_collection:
            print("⚠️ ยังไม่มี metadata ให้บันทึก")
            return ""
        out_dir = Path(Config.CORPUS_DIR)
        out_dir.mkdir(parents=True, exist_ok=True)
        stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_name = out_name or f"metadata_analysis_{stamp}.json"
        out_path = out_dir / out_name
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(self.metadata_collection, f, ensure_ascii=False, indent=2)
        print(f"✅ Exported metadata analysis: {out_path.name} ({len(self.metadata_collection)} pages)")
        return str(out_path)

    def get_usage_summary(self) -> Dict:
        """สรุปการใช้งาน"""
        return self.llm.get_usage_summary()

print("✅ Enhanced OCR Processor with Metadata ready")

✅ Enhanced OCR Processor with Metadata ready


In [10]:
# ============================================
# 📌 Block 10: Corpus Builder
# ============================================
class CorpusBuilder:
    """
    รวมไฟล์ที่อยู่ใน CLEANED_DIR ให้เป็นคอร์ปัสเดียว
    - ดึงเมทาดาต้าจากหัวไฟล์ (### ... )
    - เก็บเนื้อหา CLEANED ต่อหน้า
    - แปลงเป็น JSONL/CSV พร้อมสถิติพื้นฐาน
    """

    def __init__(self):
        self.cleaned_dir = Path(Config.CLEANED_DIR)
        self.corpus_dir = Path(Config.CORPUS_DIR)
        self.corpus_dir.mkdir(parents=True, exist_ok=True)

    def _parse_cleaned_file(self, path: Path) -> Dict:
        """อ่านไฟล์ CLEANED หนึ่งไฟล์ แล้วคืน metadata + pages"""
        text = path.read_text(encoding='utf-8', errors='ignore')
        meta = MultiPageParser.extract_metadata(text)

        # แยกหน้า (ถ้าเป็นไฟล์หลายหน้าแบบมี --- PAGE: ... ---)
        pages = MultiPageParser.parse_multipage_file(text)
        if not pages:
            # เป็น single text: ใช้ทั้งไฟล์เป็นหน้าเดียว
            pages = [{
                'page_num': 1,
                'raw_text': "",
                'cleaned_text': ThaiTextUtils.normalize_unicode(text)
            }]

        # รวมเฉพาะ CLEANED
        all_cleaned = []
        for p in pages:
            ct = p.get('cleaned_text', '')
            if ct:
                all_cleaned.append(ct)

        return {
            'file': path.name,
            'meta': meta,
            'pages': pages,
            'cleaned_joined': "\n".join(all_cleaned).strip()
        }

    def build(self, pattern: str = "*.txt", out_stem: str = None) -> Dict:
        """รวมทุกไฟล์ใน CLEANED_DIR ที่ตรง pattern เป็นคอร์ปัส"""
        files = sorted(self.cleaned_dir.glob(pattern))
        if not files:
            print("⚠️ ไม่พบไฟล์ใน CLEANED_DIR")
            return {'count': 0, 'jsonl': '', 'csv': ''}

        data_rows = []
        jsonl_rows = []
        total_chars = 0
        total_pages = 0

        for f in files:
            parsed = self._parse_cleaned_file(f)
            book = parsed['meta'].get('book_title', '')
            chapter = parsed['meta'].get('chapter', '')
            sub = parsed['meta'].get('sub_chapter', '')
            text_joined = parsed['cleaned_joined']
            n_pages = len(parsed['pages'])

            total_pages += n_pages
            total_chars += len(text_joined)

            # สำหรับ JSONL (หนึ่งไฟล์หนึ่งแถว)
            jsonl_rows.append({
                'source_file': parsed['file'],
                'book_title': book,
                'chapter': chapter,
                'sub_chapter': sub,
                'pages': n_pages,
                'text': text_joined
            })

            # สำหรับ CSV (เก็บสั้น ๆ)
            data_rows.append({
                'source_file': parsed['file'],
                'book_title': book,
                'chapter': chapter,
                'sub_chapter': sub,
                'pages': n_pages,
                'chars': len(text_joined)
            })

        stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_stem = out_stem or f"corpus_{stamp}"
        jsonl_path = self.corpus_dir / f"{out_stem}.jsonl"
        csv_path = self.corpus_dir / f"{out_stem}.csv"
        stats_path = self.corpus_dir / f"{out_stem}_stats.json"

        # เขียน JSONL
        with open(jsonl_path, "w", encoding="utf-8") as jf:
            for row in jsonl_rows:
                jf.write(json.dumps(row, ensure_ascii=False) + "\n")

        # เขียน CSV
        pd.DataFrame(data_rows).to_csv(csv_path, index=False, encoding='utf-8')

        # เขียนสถิติ
        stats = {
            'files': len(files),
            'pages': total_pages,
            'chars': total_chars,
            'avg_chars_per_file': (total_chars / len(files)) if files else 0,
            'avg_pages_per_file': (total_pages / len(files)) if files else 0,
            'created_at': datetime.now().isoformat()
        }
        with open(stats_path, "w", encoding="utf-8") as sf:
            json.dump(stats, sf, ensure_ascii=False, indent=2)

        print(f"✅ Corpus JSONL: {jsonl_path.name}")
        print(f"✅ Corpus CSV:   {csv_path.name}")
        print(f"📈 Stats:        {stats_path.name}")
        print(f"📦 Files: {stats['files']}, Pages: {stats['pages']}, Chars: {stats['chars']:,}")

        return {
            'count': len(files),
            'jsonl': str(jsonl_path),
            'csv': str(csv_path),
            'stats': stats
        }

print("✅ Corpus Builder ready")

✅ Corpus Builder ready


In [11]:
# ============================================
# 📌 Block 11: Quick Start Functions
# ============================================

def quick_setup():
    """Enhanced setup with validation"""
    print("\n🔧 Enhanced Quick Setup v3.0")
    print("=" * 50)

    # 1) ตรวจ API Keys
    if not Config.OPENAI_API_KEY:
        print("⚠️ ไม่พบ OpenAI API key")
        print(f"   กรุณาสร้างไฟล์: {Config.BASE}/openai.env หรือ openai.env.txt")
        print("   เนื้อหา: sk-xxxxxxxxxxxxxxxxxxxxxxxx")
    else:
        print("✅ OpenAI API key: OK")

    # 2) ตรวจโฟลเดอร์
    print("\n📁 Directories")
    for p in [Config.RAW_OCR_DIR, Config.CLEANED_DIR, Config.CORPUS_DIR, Config.TRAINING_PAIRS_DIR, Config.LOGS_DIR]:
        Path(p).mkdir(parents=True, exist_ok=True)
        print(f"   - {p} ✔️")

    # 3) สรุปคอนฟิก
    print("\n⚙️ Config")
    print(f"   Model: {Config.MODEL}")
    print(f"   Max pages per batch: {Config.MAX_PAGES_PER_BATCH}")
    print(f"   Temperature: {Config.TEMPERATURE}")
    print(f"   Context overlap: {Config.CONTEXT_OVERLAP}")
    print("\n✅ Setup complete.")


def quick_process_sample(filename: str):
    """
    ประมวลผลไฟล์เดียวจาก RAW_OCR_DIR โดยชื่อไฟล์ที่ระบุ
    """
    path = Path(Config.RAW_OCR_DIR) / filename
    if not path.exists():
        print(f"❌ ไม่พบไฟล์: {path}")
        return
    proc = OCRProcessor()
    result = proc.process_file(path)
    print("\n📦 Result (single):")
    print(json.dumps({k: v for k, v in result.items() if k != 'metadata'}, ensure_ascii=False, indent=2))
    return result


def quick_batch(pattern: str = "*.txt", limit: int = None):
    """
    ประมวลผลหลายไฟล์จาก RAW_OCR_DIR ด้วย pattern ที่กำหนด
    """
    proc = OCRProcessor()
    summary = proc.process_batch(file_pattern=pattern, limit=limit)
    # export training pairs (ถ้ามี)
    out_pairs = proc.export_training_pairs()
    print("\n🧾 Usage summary (session):")
    print(json.dumps(proc.get_usage_summary(), ensure_ascii=False, indent=2))
    return {'batch': summary, 'training_pairs': out_pairs}


def quick_build_corpus(pattern: str = "*.txt", out_stem: str = None):
    """
    รวมไฟล์ใน CLEANED_DIR ให้เป็นคอร์ปัส JSONL/CSV
    """
    builder = CorpusBuilder()
    return builder.build(pattern=pattern, out_stem=out_stem)


def show_usage_log_summary():
    """
    แสดงสรุป usage จากไฟล์ logs/usage.csv ทั้งหมด
    """
    logger = UsageLogger()
    s = logger.get_summary()
    print("\n📊 Usage Log Summary (all runs)")
    print(json.dumps(s, ensure_ascii=False, indent=2))
    return s

print("✅ Quick Start Functions ready")

✅ Quick Start Functions ready


In [ ]:
# ============================================
# 📌 Block 12: Main / CLI Runner (Colab-safe, with guards)
# ============================================
import argparse
from pathlib import Path

# ---------- Helpers ----------
def _fallback_usage_summary():
    """สรุป usage แบบ fallback เมื่อ UsageLogger ยังไม่พร้อม"""
    # เดา path ทั้ง 2 โหมด (Colab / Local)
    candidates = [
        Path("/content/drive/MyDrive/OCR/logs/usage.csv"),
        Path("./OCR/logs/usage.csv"),
    ]
    log_file = next((p for p in candidates if p.exists()), None)

    print("\n📒 Usage Log Summary (All time) [fallback]")
    print("=" * 50)
    if not log_file:
        print("⚠️ ไม่พบไฟล์ usage.csv ในโฟลเดอร์ logs")
        return
    try:
        import pandas as pd
        df = pd.read_csv(log_file, encoding="utf-8")
        if df.empty:
            print("ℹ️ usage.csv ว่างเปล่า")
            return
        total_files = len(df)
        total_pages = df["pages_count"].sum() if "pages_count" in df.columns else 0
        total_tokens = df["total_tokens"].sum() if "total_tokens" in df.columns else 0
        total_cost_usd = df["cost_usd"].sum() if "cost_usd" in df.columns else 0.0
        total_cost_thb = df["cost_thb"].sum() if "cost_thb" in df.columns else total_cost_usd * 35
        avg_cost_per_page = (total_cost_usd / total_pages) if total_pages else 0

        print(f"Total files         : {total_files}")
        print(f"Total pages         : {total_pages}")
        print(f"Total tokens        : {total_tokens:,}")
        print(f"Total cost (USD)    : ${total_cost_usd:.4f}")
        print(f"Total cost (THB)    : ~{total_cost_thb:.2f}")
        print(f"Avg cost per page   : ${avg_cost_per_page:.4f}")
    except Exception as e:
        print(f"⚠️ อ่าน usage.csv ไม่สำเร็จ: {e}")


def show_usage_log_summary():
    """แสดงสรุป usage โดยพยายามใช้ UsageLogger ถ้ามี ไม่งั้น fallback"""
    if "UsageLogger" in globals():
        print("\n📒 Usage Log Summary (All time)")
        print("=" * 50)
        ul = UsageLogger()
        s = ul.get_summary()
        print(f"Total files         : {s.get('total_files',0)}")
        print(f"Total pages         : {s.get('total_pages',0)}")
        print(f"Total tokens        : {s.get('total_tokens',0):,}")
        print(f"Input tokens        : {s.get('input_tokens',0):,}")
        print(f"Output tokens       : {s.get('output_tokens',0):,}")
        print(f"Total cost (USD)    : ${s.get('total_cost_usd',0):.4f}")
        print(f"Total cost (THB)    : ~{s.get('total_cost_thb',0):.2f}")
        print(f"Avg cost per page   : ${s.get('avg_cost_per_page',0):.4f}")
        print(f"Most used model     : {s.get('most_used_model','N/A')}")
        print(f"Validation stats    : {s.get('validation_stats',{})}")
    else:
        _fallback_usage_summary()


def quick_sample(sample_name: str):
    print("\n▶️ Quick Sample Mode")
    print("=" * 50)
    if not sample_name:
        print("⚠️ กรุณาระบุ --sample ชื่อไฟล์ เช่น --sample sample.txt")
        return
    if "Config" not in globals() or "OCRProcessor" not in globals():
        print("⚠️ กรุณารันบล็อก 1–11 ให้ครบก่อน (ต้องมี Config, OCRProcessor)")
        return
    file_path = Path(Config.RAW_OCR_DIR) / sample_name
    if not file_path.exists():
        print(f"❌ ไม่พบไฟล์: {file_path}")
        return
    proc = OCRProcessor()
    result = proc.process_file(file_path)
    if result.get('success'):
        print(f"\n✅ เสร็จสิ้น: {result['cleaned_path']}")
    else:
        print(f"\n❌ ล้มเหลว: {result.get('error','unknown error')}")


def quick_batch(limit: int = None):
    print("\n🚀 Batch Mode")
    print("=" * 50)
    if "Config" not in globals() or "OCRProcessor" not in globals():
        print("⚠️ กรุณารันบล็อก 1–11 ให้ครบก่อน (ต้องมี Config, OCRProcessor)")
        return
    raw_dir = Path(Config.RAW_OCR_DIR)
    files = sorted(raw_dir.glob("*.txt"))
    if not files:
        print(f"⚠️ ไม่พบไฟล์ใน {raw_dir}")
        return
    if limit is not None:
        files = files[:max(0, int(limit))]
    print(f"📁 พบไฟล์ {len(files)} ไฟล์")
    proc = OCRProcessor()
    for i, f in enumerate(files, 1):
        print(f"\n[{i}/{len(files)}] {f.name}")
        _ = proc.process_file(f)
    usage = proc.get_usage_summary()
    print("\n📊 สรุปการใช้งาน (Session)")
    print("-" * 50)
    print(f"Total tokens     : {usage['total_tokens']:,}")
    print(f"Total cost (USD) : ${usage['total_cost_usd']:.4f}")
    print(f"Total cost (THB) : ~{usage['total_cost_thb']:.2f}")
    print(f"Files processed  : {usage['session_files']}")
    print(f"Pages processed  : {usage['session_pages']}")
    print(f"Avg cost/page    : ${usage['avg_cost_per_page']:.4f}")


def quick_build_corpus(merge_name: str = None):
    print("\n🧱 Build Corpus")
    print("=" * 50)
    if "Config" not in globals():
        print("⚠️ กรุณารันบล็อก 1–11 ให้ครบก่อน (ต้องมี Config)")
        return
    cleaned_dir = Path(Config.CLEANED_DIR)
    out_dir = Path(Config.CORPUS_DIR)
    out_dir.mkdir(parents=True, exist_ok=True)
    files = sorted(cleaned_dir.glob("*.txt"))
    if not files:
        print(f"⚠️ ไม่พบไฟล์ใน {cleaned_dir}")
        return
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    corpus_name = merge_name.strip() if merge_name else f"corpus_{ts}.txt"
    out_path = out_dir / corpus_name
    print(f"📁 รวมไฟล์จาก: {cleaned_dir}")
    print(f"📝 ปลายทาง: {out_path}")
    with out_path.open("w", encoding="utf-8") as w:
        for f in files:
            try:
                txt = f.read_text(encoding="utf-8")
                w.write(f"\n\n===== FILE: {f.name} =====\n\n")
                w.write(txt)
            except Exception as e:
                print(f"⚠️ ข้ามไฟล์ {f.name}: {e}")
    print(f"✅ เสร็จสิ้น: {out_path}")


# ---------- CLI ----------
parser = argparse.ArgumentParser(description="Enhanced Thai Novel OCR Processor - Main Runner")
parser.add_argument("--sample", type=str, help="ชื่อไฟล์ใน RAW_OCR_DIR ที่ต้องการประมวลผล (เช่น sample.txt)")
parser.add_argument("--batch", action="store_true", help="ประมวลผลไฟล์ทั้งหมดใน RAW_OCR_DIR")
parser.add_argument("--limit", type=int, default=None, help="จำกัดจำนวนไฟล์ใน batch")
parser.add_argument("--build-corpus", action="store_true", help="รวมไฟล์ CLEANED ทั้งหมดเป็น corpus")
parser.add_argument("--corpus-name", type=str, default=None, help="ตั้งชื่อไฟล์ corpus เอง (เช่น my_corpus.txt)")
parser.add_argument("--summary", action="store_true", help="แสดงสรุป usage log สะสมทั้งหมด")

# ใช้ parse_known_args กัน -f kernel.json ใน Colab
args, _ = parser.parse_known_args()

def _main():
    ran_any = False
    if args.sample:
        ran_any = True
        quick_sample(args.sample)
    if args.batch:
        ran_any = True
        print("\n📦 โหมด Batch")
        quick_batch(limit=args.limit)
    if args.build_corpus:
        ran_any = True
        print("\n📚 รวม Corpus")
        quick_build_corpus(merge_name=args.corpus_name)
    if args.summary or not ran_any:
        if not ran_any:
            print("\nℹ️ ไม่ส่ง argument ใด ๆ จะแสดงสรุปให้ก่อน")
        print("\n📈 สรุปการใช้งาน")
        show_usage_log_summary()

if __name__ == "__main__":
    _main()


ℹ️ ไม่ส่ง argument ใด ๆ จะแสดงสรุปให้ก่อน

📈 สรุปการใช้งาน

📒 Usage Log Summary (All time)
Total files         : 0
Total pages         : 0
Total tokens        : 0
Input tokens        : 0
Output tokens       : 0
Total cost (USD)    : $0.0000
Total cost (THB)    : ~0.00
Avg cost per page   : $0.0000
Most used model     : N/A
Validation stats    : {}


In [12]:
# 🚀 Run me
processor = OCRProcessor(analyze_metadata=True)  # True = มี metadata, False = แค่ clean
result = processor.process_file(Path(Config.RAW_OCR_DIR) / "Namiya.txt")

print("\n📊 Run Summary:")
print("Success:", result.get('success'))
if result.get('success'):
    print("Output file:", result.get('cleaned_path'))
    print("Cost: $", result.get('cost'))
    print("Tokens used:", result.get('tokens'))

show_usage_log_summary()

✅ OpenAI client ready (Model: gpt-4o-mini)

📄 Processing: Namiya.txt
   🔍 Metadata analysis: ENABLED
   📖 Multi-page file detected
   📊 Found 5 pages
   📘 Book: (ปาฏิหารย์ร้านชำของคุณนามิยะ)
   🧾 Chapter: (บทที่ 2 เสียงหีบเพลงปากในยามดึกสงัด 2-8)
   📏 Total content: 4,250 chars (~3,188 tokens)
   🚀 Processing all 5 pages in one batch...
   ⚠️ Warning: ไม่พบผลลัพธ์สำหรับหน้า 1
   ⚠️ Warning: ไม่พบผลลัพธ์สำหรับหน้า 2
   ⚠️ Warning: ไม่พบผลลัพธ์สำหรับหน้า 3
   ⚠️ Warning: ไม่พบผลลัพธ์สำหรับหน้า 4
   ⚠️ Warning: ไม่พบผลลัพธ์สำหรับหน้า 4
   🔍 Analyzing metadata for each page...
      Analyzing page 1...
      Analyzing page 2...
      Analyzing page 3...
      Analyzing page 4...
      Analyzing page 4...
   ✅ Metadata analysis complete: 4,992 tokens, $0.0012
   ✅ Saved: Namiya_clean_20250829_134344.txt
   📄 Pages: 5
   🔤 Tokens: 9,301
   ⏱ Time: 32.0s
   💰 Cost: $0.0027 (~0.09 บาท)
   📋 Validation: ✅0 ⚠️5 ❌0 (avg: 0.90)
   🔍 Metadata analyzed: 4 pages

📊 Run Summary:
Success: True
Output f

TypeError: Object of type int64 is not JSON serializable